In [1]:
import re, time, datetime as dt, requests
from bs4 import BeautifulSoup, UnicodeDammit, FeatureNotFound
import pandas as pd
import lxml

BASE = 'https://npb.jp/bis/{year}/games/'

def soup_from(url: str, *, timeout: int = 10):
    """文字コードを自動判定して BeautifulSoup を返す"""
    # ① バイト列で取得
    html_bytes = requests.get(url, timeout=timeout).content
    
    # ② UnicodeDammit が <meta charset> と BOM を解析して最適な decode を実施
    html_str = UnicodeDammit(
        html_bytes,
        is_html=True,
        smart_quotes_to="utf-8"   # 変な全角カギ括弧を避けたい時に便利
    ).unicode_markup
    
    # ③ Soup 化（lxml が無ければ標準パーサにフォールバック）
    try:
        return BeautifulSoup(html_str, "lxml")
    except FeatureNotFound:
        return BeautifulSoup(html_str, "html.parser")

def list_game_urls(date: dt.date):
    """その日の全試合ページ URL を返す"""
    idx = f'{BASE.format(year=date.year)}gm{date:%Y%m%d}.html'
    soup = BeautifulSoup(requests.get(idx).content, 'lxml')
    pat  = re.compile(r'^[sp]\d{13}\.html$')           # s2024080101323.html など
    hrefs = [a['href'] for a in soup.select('a[href]') if pat.match(a['href'])]
    return [BASE.format(year=date.year) + h for h in hrefs]

def parse_game(url: str) -> dict:
    """BIS 個別試合ページ → {date, team_away, team_home, R_away, R_home}"""
    soup = soup_from(url)

    # ========== チーム名 ==========
    # ① 新レイアウト (2023‑) : #gmdivscore 内 contentshdname
    names = [td.get_text(strip=True)              # 中日ドラゴンズ / 東京ヤクルトスワローズ
             for td in soup.select('#gmdivscore td.contentshdname')]
    #      0: home, 1: away なので一旦保持
    # ② 旧レイアウト (2022 以前) : table.linescore → .teamLeft/.teamRight
    if len(names) != 2:
        alt1 = soup.select_one('.teamLeft, .teamNameL')
        alt2 = soup.select_one('.teamRight, .teamNameR')
        if alt1 and alt2:
            names = [alt2.get_text(strip=True), alt1.get_text(strip=True)]  # away, home
    if len(names) != 2:
        raise RuntimeError("チーム名が取れませんでした")


    # ========== 得点 ==========
    # 新レイアウトは #gmdivscore 内 gmboxrun（home → away）
    runs = [int(td.get_text(strip=True)) 
            for td in soup.select('#gmdivscore td.gmboxrun')]
    # 旧レイアウトは table.linescore tr.total td（visitor → home）
    if len(runs) != 2:
        runs = [int(td.get_text(strip=True))
                for td in soup.select('table.linescore tr.total td')[:2]]

    if len(runs) != 2:
        # table.gmscoreteam から総得点列 (R) をつかむ最後の保険
        rows = soup.select('#gmdivresult tr')[1:3]     # visitor → home
        if rows and all(r.select_one('td.gmscoreteam') for r in rows):
            runs = [int(r.find_all('td', class_='gmscore')[-3].text) for r in rows]

    if len(runs) != 2:
        raise RuntimeError("得点が取れませんでした")

    # ――― レイアウト別に home/away 並びが逆になるので合わせる ―――
    # names が [home, away] で runs も [home, away] になっていれば swap
    # ↓ stadium の “@” マークはないので、gmdivresult の行数（visitor → home）で判定する
    away_first = soup.select_one('#gmdivresult td.gmscoreteam')
    if away_first and away_first.get_text(strip=True) == names[1]:
        # names = [home, away] なので並び替え
        names  = [names[1],  names[0]]
        runs   = [runs[1],   runs[0]]

    ymd = re.search(r'(\d{8})', url).group(1)          # 20240801
    return {
        "date"     : dt.datetime.strptime(ymd, "%Y%m%d").date(),
        "team_away": names[0],
        "team_home": names[1],
        "R_away"   : runs[0],
        "R_home"   : runs[1],
    }

def season_games(year=2024, start=dt.date(2024,3,29), end=dt.date(2024,4,1)):
    """1シーズン分を走査して DataFrame へ"""
    cur, records = start, []
    while cur <= end:
        for g in list_game_urls(cur):
            try:
                records.append(parse_game(g))
                time.sleep(0.2)           # polite crawl
            except Exception as e:        # 試合中止ページなどは飛ばす
                print(f'skip {g} : {e}')
        cur += dt.timedelta(days=1)
    return pd.DataFrame(records)




# df = season_games(year=2024, start=dt.date(2024,3,29), end=dt.date(2024,10,9))


In [15]:
df = season_games(year=2020, start=dt.date(2020,6,19), end=dt.date(2020,11,11))

skip https://npb.jp/bis/2020/games/s2020063000115.html : invalid literal for int() with base 10: ''
skip https://npb.jp/bis/2020/games/s2020070300125.html : invalid literal for int() with base 10: ''
skip https://npb.jp/bis/2020/games/s2020070601379.html : invalid literal for int() with base 10: ''
skip https://npb.jp/bis/2020/games/s2020070700133.html : invalid literal for int() with base 10: ''
skip https://npb.jp/bis/2020/games/s2020070700134.html : invalid literal for int() with base 10: ''
skip https://npb.jp/bis/2020/games/s2020070800136.html : invalid literal for int() with base 10: ''
skip https://npb.jp/bis/2020/games/s2020070900228.html : invalid literal for int() with base 10: ''
skip https://npb.jp/bis/2020/games/s2020071000141.html : invalid literal for int() with base 10: ''
skip https://npb.jp/bis/2020/games/s2020071400241.html : invalid literal for int() with base 10: ''
skip https://npb.jp/bis/2020/games/s2020080200713.html : invalid literal for int() with base 10: ''


In [14]:
import pandas as pd

def check_npb_2024_completeness(df: pd.DataFrame) -> None:
    """
    引数
    ----
    df : columns = ['date', 'team_away', 'team_home', ...] を想定
         1 行＝1 試合

    返り値
    ------
    なし（print でレポート）
    """

    # -----------------------------------------------
    # 1. 基本統計
    # -----------------------------------------------
    print("【行数】", len(df))

    # 同一日・同一カードの重複行が無いか
    dup_mask = df.duplicated(subset=["date", "team_away", "team_home"], keep=False)
    n_dup = dup_mask.sum()
    print("【重複行】", n_dup)

    if n_dup:
        print(df.loc[dup_mask, ["date", "team_away", "team_home"]])

    # -----------------------------------------------
    # 2. チーム別試合数を数える
    # -----------------------------------------------
    # away / home の列を縦に結合して集計
    counts = (
        pd.concat(
            [
                df[["team_away"]].rename(columns={"team_away": "team"}),
                df[["team_home"]].rename(columns={"team_home": "team"}),
            ],
            axis=0,
            ignore_index=True,
        )
        .value_counts(subset=["team"])
        .sort_index()
        .rename("games")
        .to_frame()
    )

    # 2024 年レギュラーシーズンの規定試合数
    EXPECTED = 143

    counts["OK?"] = counts["games"] == EXPECTED
    print("\n【チーム別出場試合数】")
    print(counts)

    # -----------------------------------------------
    # 3. 全チームが揃っているか
    # -----------------------------------------------
    n_ok  = counts["OK?"].sum()
    n_all = counts.shape[0]

    print(f"\n【判定】 {n_ok}/{n_all} チームが {EXPECTED} 試合を消化")

    # 総試合数チェック（143 試合×12 球団÷2）
    expected_total = EXPECTED * n_all // 2
    if len(df) == expected_total and n_ok == n_all and n_dup == 0:
        print("✅ データセットは完全です")
    else:
        print("⚠️ 取りこぼし / ダブりがある可能性があります")

# ---------------------------------------------------
# 使い方
# ---------------------------------------------------
check_npb_2024_completeness(df)


【行数】 717
【重複行】 0

【チーム別出場試合数】
                games    OK?
team                        
オリックス・バファローズ      120  False
中日ドラゴンズ           120  False
北海道日本ハムファイターズ     119  False
千葉ロッテマリーンズ        120  False
埼玉西武ライオンズ         120  False
広島東洋カープ           119  False
東京ヤクルトスワローズ       120  False
東北楽天ゴールデンイーグルス    119  False
横浜DeNAベイスターズ      119  False
福岡ソフトバンクホークス      120  False
読売ジャイアンツ          119  False
阪神タイガース           119  False

【判定】 0/12 チームが 143 試合を消化
⚠️ 取りこぼし / ダブりがある可能性があります


In [13]:
df.to_csv('games_2020.csv')